In [ ]:
#!/usr/bin/env python
# make sure to install these packages before running:
# pip install pandas
# pip install sodapy
# pip install seaborn

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sodapy import Socrata


In [ ]:
# Descargar datos del RUNT 2.0
client = Socrata('www.datos.gov.co', None)
results = client.get('u3vn-bdcy', where='anio >= 2023 AND anio <= 2025', limit=500000)
df = pd.DataFrame.from_records(results)
print('Columnas disponibles:', df.columns.tolist())


In [ ]:
# Preparar columnas principales
anio_col = next(c for c in df.columns if 'anio' in c.lower())
mes_col = next(c for c in df.columns if 'mes' in c.lower())
mun_col = next(c for c in df.columns if 'muni' in c.lower())
dep_col = next(c for c in df.columns if 'depart' in c.lower())
clase_col = next(c for c in df.columns if 'clase' in c.lower())
cantidad_col = next((c for c in df.columns if 'cantidad' in c.lower() or 'total' in c.lower()), None)
if cantidad_col is None:
    raise ValueError('No se encontró columna de cantidad')

# Convertir tipos numéricos y fecha
for col in [anio_col, mes_col, cantidad_col]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df['fecha'] = pd.to_datetime(df[anio_col].astype(int).astype(str) + '-' + df[mes_col].astype(int).astype(str) + '-01')
df['quarter'] = df['fecha'].dt.to_period('Q')

# Limitar a los tipos de transporte con mayor representatividad
clase_top = (df.groupby(clase_col)[cantidad_col]
               .sum()
               .sort_values(ascending=False)
               .head(5)
               .index)
df = df[df[clase_col].isin(clase_top)]

df.head()


In [ ]:
def plot_scope(df, filter_fn, scope_name):
    scope_df = df[filter_fn(df)]
    grouped = (scope_df.groupby(['quarter', clase_col])[cantidad_col]
                        .sum()
                        .reset_index())
    grouped['quarter'] = grouped['quarter'].astype(str)
    plt.figure(figsize=(12,6))
    sns.lineplot(data=grouped, x='quarter', y=cantidad_col, hue=clase_col, marker='o')
    plt.title(f'Crecimiento del parque automotor - {scope_name}')
    plt.xlabel('Trimestre')
    plt.ylabel('Cantidad de vehículos')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [ ]:
# Gráficas para Cali, Valle y nivel nacional
plot_scope(df, lambda d: d[mun_col].str.upper().str.contains('CALI'), 'Cali')
plot_scope(df, lambda d: d[dep_col].str.upper() == 'VALLE DEL CAUCA', 'Valle del Cauca')
plot_scope(df, lambda d: d[mun_col].notna(), 'Colombia')
